In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load and clean data
df = pd.read_csv('/content/drive/MyDrive/IMDb Movies India.csv', encoding='latin1')
df = df[['Name', 'Genre', 'Rating']].dropna()
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df = df.dropna(subset=['Rating']).reset_index(drop=True)

# TF-IDF on Genre
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(df['Genre'])

# Target variable
y = df['Rating']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(f"RMSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")

# Function to predict rating of a specific movie
def predict_rating(movie_name):
    row = df[df['Name'].str.lower() == movie_name.lower()]
    if row.empty:
        return f"'{movie_name}' not found in dataset."

    genre_text = row.iloc[0]['Genre']
    genre_vector = tfidf.transform([genre_text])
    predicted_rating = model.predict(genre_vector)[0]

    return f"Predicted rating for '{movie_name}': {predicted_rating:.2f}"

# Example usage
print(predict_rating("Krrish"))

RMSE: 1.57
R² Score: 0.13
Predicted rating for 'Krrish': 5.35


In [9]:
!pip install gradio --quie

In [23]:
import gradio as gr

def gradio_predict(movie_name):
    return predict_rating(movie_name)

interface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(lines=1, placeholder="Enter movie name...", label="Movie Title"),
    outputs=gr.Textbox(label="Predicted Rating"),
    title="🎬 Movie Rating Predictor",
    description="Enter the name of a movie to predict its rating based on its genre using Linear Regression.",
    examples=["Krrish", "3 Idiots", "Dangal"]
)

interface.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://299e0d140c81c7ede3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
